# Hydroelastic Contact: Nonconvex Meshes
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).

If you are not familiar with Drake's hydroelastic contact, study [hydroelastic_contact_basics.ipynb](./hydroelastic_contact_basics.ipynb). You can also find more information in Hydroelastic Contact User Guide [here.](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

## Introduction

This tutorial shows you how to set up dynamic simulations with hydroelastic contacts using nonconvex meshes. For context, we use a simple example of a compliant hydroelastic bell pepper dropped onto a compliant hydroelastic bowl on a compliant hydroelastic table top. Contact forces are calculated and visualized.

## Start MeshCat

See the section [Viewing models](./authoring_multibody_simulation.ipynb#Viewing-models) in the tutorial [Authoring a Multibody Simulation](./authoring_multibody_simulation.ipynb) for an introduction to MeshCat.

In [ ]:
from pydrake.geometry import StartMeshcat

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

## Create compliant-hydroelastic bell pepper in SDFormat

*Make sure you have the MeshCat tab opened in your browser;
the link is shown immediately above.*

We will create and visualize a compliant-hydroelastic bell pepper with SDFormat string.

We will use `ModelVisualizer` to verify the SDFormat description.
It will tell MeshCat to show the bell pepper.

### Visual geometry

First we create a visual geometry of the bell pepper. Observe in MeshCat the X(red), Y(green), and Z(blue) axes have origin near the bottom of the bell pepper. In general, a mesh geometry can have its origin anywhere.

Here MeshCat can show only the `drake/illustration` without `drake/proximity`.

In [ ]:
from pydrake.visualization import ModelVisualizer

visual_bell_pepper_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="BellPepper">
    <pose>0 0 0 0 0 0</pose>
    <link name="bell_pepper">
      <visual name="yellow_bell_pepper_no_stem">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(visual_bell_pepper_sdf, "sdf")
visualizer.Run(loop_once=True)

##### Collision geometry with hydroelastic properties

We will add the `<collision>` block with hydroelastic properties. Without `<collision>`, our object cannot make contact in the simulation.

Drake also uses the term proximity for collision. This is the `<drake:proximity_properties>` block that control hydroelastic contacts for SDFormat:

        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e6</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
        </drake:proximity_properties>

Use `<drake:compliant_hydroelastic/>` to specify a compliant-hydroelastic object.

We set `<drake:hydroelastic_modulus>` to 1e6 Pascals. It is comparable to high-density polyethylene (HDPE) with 1 GPa Young's modulus. Hydroelastic modulus is not a physical property but rather a numerical parameter to tune our contact model. As a rule of thumb, set hydroelastic modulus to about 1/100 of Young's modulus. 

We set `<drake:mu_dynamic>` (unitless) to 0.5 for the coefficient of dynamic (i.e., kinetic) friction, and set `<drake:hunt_crossley_dissipation>` to 1.25 seconds/meter as the dissipation constant for the Hunt & Crossley model.

Here MeshCat can show both `drake/illustration` and `drake/proximity` but `drake/inertia` is wrong. Toggle `drake/illustration` and `drake/proximity`on and off.

In [ ]:
from pydrake.visualization import ModelVisualizer

collision_bell_pepper_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="BellPepper">
    <pose>0 0 0 0 0 0</pose>
    <link name="bell_pepper">
      <collision name="collision">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.vtk</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e6</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25 </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
      <visual name="visual">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(collision_bell_pepper_sdf, "sdf")
visualizer.Run(loop_once=True)

### Inertial properties

In order to simulate dynamics, we will add `<mass>` of 0.159kg and `<inertia>` matrix.

See the helpful pydrake tools [fix_inertia](https://drake.mit.edu/pydrake/pydrake.multibody.fix_inertia.html) and [mesh_to_model](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html) to compute the inertia matrix.

The function `bell_pepper()` below returns SDFormat description given `hydroelastic_modulus`. We will use this function later when we experiment with different values of hydroelastic modulus.

Notice that the parameter `hydroelastic_modulus` is a character string `'1e6'` (it is enclosed by single quote `'`), not a floating-point number `1e6`, so that the SDFormat description does not lose precision due to formatting.

After running the next block, MeshCat will show `drake/illustration`, `drake/proximity`, and the correct `drake/inertia`.

In [ ]:
from pydrake.visualization import ModelVisualizer

# Define a compliant-hydroelastic bell pepper from
# a given hydroelastic modulus.
def bell_pepper(hydroelastic_modulus='1e6'):
    return f"""<?xml version="1.0"?>
    <sdf version="1.7">
      <model name="BellPepper">
        <pose>0 0 0 0 0 0</pose>
        <link name="bell_pepper">
          <inertial>
            <pose>0.000537 -0.00272 0.0384 0 0 0</pose>
            <mass>0.159</mass>
            <inertia>
              <ixx> 0.000101</ixx>
              <ixy>-0.000001</ixy>
              <ixz>-0.000004</ixz>
              <iyy> 0.000105</iyy>
              <iyz> 0.000007</iyz>
              <izz> 0.000107</izz>
            </inertia>
          </inertial>
          <collision name="collision">
            <geometry>
              <mesh>
                <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.vtk</uri>
                <scale>1 1 1</scale>
              </mesh>
            </geometry>
            <drake:proximity_properties>
              <drake:compliant_hydroelastic/>
              <drake:hydroelastic_modulus>{hydroelastic_modulus}</drake:hydroelastic_modulus>
              <drake:mu_dynamic>0.5</drake:mu_dynamic>
              <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
            </drake:proximity_properties>
          </collision>
          <visual name="visual">
            <geometry>
              <mesh>
                <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.obj</uri>
                <scale>1 1 1</scale>
              </mesh>
            </geometry>
            <material>
              <diffuse>1.0 1.0 1.0 0.5</diffuse>
            </material>
          </visual>
        </link>
      </model>
    </sdf>
    """

bell_pepper_sdf = bell_pepper()

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(bell_pepper_sdf, "sdf")
visualizer.Run(loop_once=True)

## Create compliant-hydroelastic table top in URDF

The following URDF string specifies a compliant-hydroelastic box for a table top.

Both the `<visual>` and `<collision>` geometries are boxes of size 60cm x 1m x 5cm. Observe in MeshCat the X(red), Y(green), and Z(blue) axes.


We do not specify `<mass>` and `<inertia>` of the table top because, in the next section when we set up `Diagram`, we will fix the table top to the world frame. It will not move.

The `<frame name="top_surface">` is a frame at the top of the table top with the tag:

    <pose relative_to="table_top_link">0 0 0.025 0 0 0</pose>

It is at 2.5cm (half of the box's height) above the center of the box. In the next section, we will create a scene that places the `top_surface` frame at the world's origin.

In [ ]:
from pydrake.visualization import ModelVisualizer

# Create a rigid-hydroelastic table top
table_top_urdf = """<?xml version="1.0"?>
  <robot name="TableTop">
    <link name="table_top_link">
      <visual name="visual">
        <geometry>
          <box size="0.6 1.0 0.05"/>
        </geometry>
        <material>
         <color rgba="0.9 0.8 0.7 0.5"/>
        </material>
      </visual>
      <collision name="collision">
        <geometry>
          <box size="0.6 1.0 0.05"/>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus value="1e7"/>
          <drake:mu_dynamic value="0.5"/>
          <drake:hunt_crossley_dissipation value="1.25"/>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="top_surface" link="table_top_link" xyz="0 0 0.025" rpy="0 0 0"/>
  </robot>
"""

# Visualize the URDF string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(table_top_urdf, "urdf")
visualizer.Run(loop_once=True)

## Create compliant-hydroelastic bowl in URDF

In [ ]:
bowl_urdf = """<?xml version="1.0"?>
  <robot name="Bowl">
    <link name="bowl">
      <inertial>
        <mass value="0.5"/>
        <origin xyz="0 0 -0.00613"/>
        <inertia ixx="0.00389"
                 ixy="0.00000"
                 ixz="0.00000"
                 iyy="0.00392"
                 iyz="0.00000"
                 izz="0.00637"/>
      </inertial>
      <visual name="visual">
        <geometry>
          <mesh filename="package://drake_models/dishes/bowls/evo_bowl_no_mtl.obj"/>
        </geometry>
        <material>
         <color rgba="0.9 0.8 0.7 0.5"/>
        </material>
      </visual>
      <collision name="collision">
        <geometry>
          <mesh filename="package://drake_models/dishes/bowls/evo_bowl.vtk"/>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus value="1e7"/>
          <drake:mu_dynamic value="0.5"/>
          <drake:hunt_crossley_dissipation value="1.25"/>
        </drake:proximity_properties>
      </collision>
    </link>
  </robot>
"""

# Visualize the URDF string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(bowl_urdf, "urdf")
visualizer.Run(loop_once=True)

## Create Diagram of the scene

The function `add_scene()` below creates a scene using the assets that we created.

It uses `DiagramBuilder` to create `MultibodyPlant` and `SceneGraph`. It uses `Parser` to add the two SDFormat strings of the two boxes into `Diagram` of the scene.

It sets `discrete_contact_approximation` to `"similar"` for stability during high impacts. See [DiscreteContactApproximation](https://drake.mit.edu/doxygen_cxx/namespacedrake_1_1multibody.html#:~:text=enum%20DiscreteContactApproximation).

It fixes table's top surface to the world frame by calling `WeldFrames()` and places the bell pepper 30 centimeters above the table top.

After this step, the next section will add visualization to `DiagramBuilder`.

In [ ]:
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.framework import DiagramBuilder

def clear_meshcat():
    # Clear MeshCat window from the previous blocks.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

def add_scene(time_step):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_approximation="similar"),
        builder)
    parser = Parser(plant)

    # Load the table top and the box we created.
    parser.AddModelsFromString(table_top_urdf, "urdf")
    parser.AddModelsFromString(bowl_urdf, "urdf")
    parser.AddModelsFromString(bell_pepper_sdf, "sdf")

    # Weld the table top to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Set initial position of the bowl
    X_WB = RigidTransform(p=[0, 0, 0.061])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("bowl"), X_WB)
    
    # Set how high the center of the bell pepper is from the world's origin. 
    # W = the world's frame
    # C = frame at the center of the bell pepper
    X_WC = RigidTransform(p=[0, 0, 0.301])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("bell_pepper"), X_WC)

    return builder, plant

## Set up visualization of the simulation

The function `add_scene()` above does not have visualization, so we will do it in the function `add_viz()` below.

It uses `DiagramBuilder` from `add_scene()`. The `meshcat` interface represents the same MeshCat window that we have been using.

We set the `publish_period` to 1/64 to publish 64 frames per simulated second. In this example, we publish frequently so we can better see changes in contact. See documentation of [VisualizationConfig](https://drake.mit.edu/doxygen_cxx/structdrake_1_1visualization_1_1_visualization_config.html) for more details.

At this step we set `publish_contacts` to `False`, we will set up contact visualization near the end of this tutorial.

In [ ]:
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

def add_viz(builder, plant):
    ApplyVisualizationConfig(
        config=VisualizationConfig(
                   publish_period = 1 / 32.0,
                   publish_contacts = False),
        builder=builder, meshcat=meshcat)
    
    return builder, plant

## Visualize contact results

To visualize contact results, we will add `ContactVisualizer` to `Diagram` of the previous section.

The following function `add_contact_viz()` adds `ContactVisualizer` to `DiagramBuilder` using `MultibodyPlant` and `Meshcat`.

With `newtons_per_meter= 2e1`, it will draw a red arrow of length 1 meter for each force of 20 newtons. With `newtons_meters_per_meter= 1e-1`, it will draw a blue arrow of length 1 meter for each torque of 0.1 newton\*meters. The next section will run the simulation.

In [ ]:
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams


def add_contact_viz(builder, plant):
    contact_viz = ContactVisualizer.AddToBuilder(
        builder, plant, meshcat,
        ContactVisualizerParams(
            publish_period= 1.0 / 32.0,
            newtons_per_meter= 2e1,
            newton_meters_per_meter= 1e-1))

    return builder, plant

### Run simulation with contact visualization

The following code will run the simulation. In MeshCat, the red arrow represents the force `f`, and the blue arrow represents the torque `tau`. You should see the contact patch moving around together with the force and torque vectors. At the end of simulation, it will also report contact result numerically.

Notice that the code below does not publish the recording yet. The next section will publish the recording for playback.

In [ ]:
from pydrake.systems.analysis import Simulator

def run_simulation_with_contact_viz(sim_time, time_step=1e-2):
    clear_meshcat()
    
    builder, plant = add_scene(time_step)
    add_viz(builder, plant)
    add_contact_viz(builder, plant)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    
    meshcat.StartRecording(frames_per_second=64.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()

    
run_simulation_with_contact_viz(sim_time=2)

### Playback recording to appreciate dynamics

After running the code below, playback with `timeScale` = 0.1 or 0.01 to appreciate the contact dynamics. You should see the force and torque vectors oscillate synchronously with the rocking bell pepper.

Currently playing back the simulation will show contact force and moment; however, it does not show contact patch appropriately, which could be confusing. It shows the last recorded contact patch. Issue [19142](https://github.com/RobotLocomotion/drake/issues/19142) explains the problem in more details.

In [ ]:
meshcat.PublishRecording()

## Optional exercises

The following sections are optional. It strengthens your understanding of our code and its capability, but you do not need to learn it in order to use hydroelastic contact.

### Download simulation result into a html file for sharing

You can download the simulation result into a self-contained html file, allowing others to playback the simulated results without simulating. The following code prints the URL for downloading. Click on the printed URL to download.

In [ ]:
print(f"{meshcat.web_url()}/download")

### High impact at first contact
At 0.25 second, the bell pepper makes a high impact on the table top. Remove `#` below and run the code.You should see a very long red arrow representing a very large force!

In [ ]:
# run_simulation_with_contact_report_and_viz(sim_time=0.25)
# meshcat.PublishRecording()

### Effects of low hydroelastic modulus

The code below change the value of hydroelastic modulus in `bell_pepper_sdf` to the `new_hydroelastic_modulus`. Notice that we use the single quoted `'` to enclose the number for `new_hydroelastic_modulus`, so it is treated as a character string.

Set `new_hydroelastic_modulus` to unphysically low value like 1e2 Pascals, and you should see the bell pepper drops down and passes through the tabel top. This is because the low hydroelastic modulus cannot make enough contact force to match gravity force. Most materials has Young's modulus in the range of 1e6 Pa (rubber) to 1e12 Pa (carbon nanotube), so their hydroelastic modulus should be in the range of 1e5 Pa (rubber) to 1e10 Pa (carbon nanotube).

Remove `#` in the code below and run it.

Playback slowly with timeScale 0.1 in MeshCat, and you should see a very short red arrow of contact force around 0.25 seconds.

In [ ]:
# new_hydroelastic_modulus = '1e2'
# bell_pepper_sdf = bell_pepper(hydroelastic_modulus = new_hydroelastic_modulus)
# 
# run_simulation_with_contact_report_and_viz(sim_time=0.27)
# meshcat.PublishRecording()

## Further reading

* [Hydroelastic Contact User Guide](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

* Elandt, R., Drumwright, E., Sherman, M., & Ruina, A. (2019, November). A pressure field model for fast, robust approximation of net contact force and moment between nominally rigid objects. In 2019 IEEE/RSJ International Conference on Intelligent Robots and Systems(IROS) (pp. 8238-8245). IEEE. [link](https://arxiv.org/abs/1904.11433)

* Masterjohn, J., Guoy, D., Shepherd, J., & Castro, A. (2022). Velocity Level Approximation of Pressure Field Contact Patches. IEEE Robotics and Automation Letters 7, no. 4 (2022): 11593-11600. [link](https://arxiv.org/abs/2110.04157v2)

* Elandt, R. (2022, December). Pressure Field Contact. Dissertation. Cornell University. [link](https://ecommons.cornell.edu/handle/1813/112919)